In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ISR Dataset/IMDB Dataset.csv')

data['review'] = data['review'].str.lower()


In [ ]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because",
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should",
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [ ]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


In [ ]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

In [ ]:
data_without_stopwords.head()

,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [ ]:

reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically family little boy  jake  thinks zomb...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  wasn t creative ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

In [ ]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])



In [ ]:
sentiment = data_without_stopwords['sentiment']

In [ ]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [ ]:
len(Y_train)

40000

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [ ]:
words_to_index = tokenizer.word_index

In [ ]:
len(words_to_index)

95419

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map


In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/ISR Dataset/glove.6B.50d.txt')

In [ ]:
maxLen = 150


In [ ]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)






In [ ]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)

  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)

  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model








In [ ]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          91648     
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584

In [ ]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 256)           1968

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 214s 341ms/step - loss: 0.6198 - accuracy: 0.6440
Epoch 2/15
625/625 [==============================] - 209s 334ms/step - loss: 0.4746 - accuracy: 0.7732
Epoch 3/15
625/625 [==============================] - 206s 330ms/step - loss: 0.4209 - accuracy: 0.8074
Epoch 4/15
625/625 [==============================] - 206s 330ms/step - loss: 0.3887 - accuracy: 0.8233
Epoch 5/15
625/625 [==============================] - 206s 329ms/step - loss: 0.3648 - accuracy: 0.8363
Epoch 6/15
625/625 [==============================] - 206s 329ms/step - loss: 0.3378 - accuracy: 0.8534
Epoch 7/15
625/625 [==============================] - 205s 328ms/step - loss: 0.3134 - accuracy: 0.8639
Epoch 8/15
625/625 [==============================] - 204s 326ms/step - loss: 0.2878 - accuracy: 0.8780
Epoch 9/15
625/625 [==============================] - 204s 327ms/step - loss: 0.2619 - accuracy: 0.8923
Epoch 10/15
625/625 [==============================] - 208s 332m

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 583s 926ms/step - loss: 0.6276 - accuracy: 0.6233
Epoch 2/15
625/625 [==============================] - 581s 929ms/step - loss: 0.4958 - accuracy: 0.7641
Epoch 3/15
625/625 [==============================] - 581s 929ms/step - loss: 0.4656 - accuracy: 0.7781
Epoch 4/15
625/625 [==============================] - 575s 920ms/step - loss: 0.4297 - accuracy: 0.8013
Epoch 5/15
625/625 [==============================] - 570s 912ms/step - loss: 0.4078 - accuracy: 0.8135
Epoch 6/15
625/625 [==============================] - 576s 922ms/step - loss: 0.3891 - accuracy: 0.8241
Epoch 7/15
625/625 [==============================] - 572s 916ms/step - loss: 0.3779 - accuracy: 0.8324
Epoch 8/15
625/625 [==============================] - 584s 934ms/step - loss: 0.3706 - accuracy: 0.8357
Epoch 9/15
625/625 [==============================] - 575s 920ms/step - loss: 0.3706 - accuracy: 0.8371
Epoch 10/15
625/625 [==============================] - 574s 918m

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 46s 140ms/step - loss: 0.3548 - accuracy: 0.8462


[0.3547525703907013, 0.8461999893188477]

In [ ]:
model_1d.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 15s 46ms/step - loss: 0.3663 - accuracy: 0.8362


[0.3663209080696106, 0.8361999988555908]

In [ ]:
preds = model_1d.predict(X_test_indices)

In [ ]:
n = np.random.randint(0,9999)

X_test[n]


'three main problems film  rather three reasons isn t even contender worthy serious consideration firstly  always going true  not nearly good books  however  least expected reflect sapkowski s wit depth secondly production  fantasy movies are  opinion  hardest produce well  everything props costumes  scenery  stunts  especially  cgi substandard finally  many subjective  just can t quite take acting seriously  wasn t brought poland can t really judge  may lack familiarity films polish  lines seem amateurishly delivered    casting better well in summation  possible redeeming feature film remainder plot shining through  not unlike diamond ring rotting cadaver on separate note heard books coming english  haven t seen yet can t imagine one even begin translate them    ask english speaking reader bare mind judging book '

In [ ]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else:
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


precicted sentiment : negative
correct sentiment : negative


In [ ]:
preds[n]

array([0.46030152], dtype=float32)

In [ ]:
preds1 = model.predict(X_test_indices)

In [ ]:
n = np.random.randint(0,9999)

X_test[n]


'artistic standpoint  movie fails entertain provide moral resolution  plot hard follow  one all  acting tough stomach  dialogue cliche clearly written somebody doesn t know english  cinematography worth watching movie  although can still see flaws picture as asian american female  movie angered proved narrow minded people can be  politically speaking  putting asian women disgraceful roles outlawed  perpetuating stereotypes asian prostitutes not bad self esteem asian women  also country whole  feel lead actress played maya exploitation must faced playing role as americans  please avoid movie  art used develop tolerance respect one another  not hold back '

In [ ]:
if preds1[n] > 0.5:
  print('predicted sentiment : positive')
else:
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


precicted sentiment : negative
correct sentiment : negative


In [ ]:
preds1[n]

array([0.02735806], dtype=float32)

In [ ]:
Y_test[n]

0

In [ ]:
model_1d.save_weights('/content/drive/MyDrive/ISR Dataset/imdb_weights_con1vd.hdf5')

In [ ]:
model.save_weights('/content/drive/MyDrive/ISR Dataset/imdb_weights_lstm.hdf5')

In [ ]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [ ]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data



In [ ]:
data = add_score_predictions(data, reviews_list_idx)

In [ ]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.763237,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.981091,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.981353,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.017200,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.988156,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.847537,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.006484,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.435211,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.840934,positive
